In [1]:
import os, sys
import io
import requests
import random
import tkinter as tk
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter
from ipywidgets import IntProgress
from IPython.display import display
from PIL import Image, ImageTk

import skimage as ski
import cv2
import torch
import piq
import nvidia_smi

# local library of functions to connect to image server
import libutils.use_server as server

In [2]:
img_path = server.init() 

server : http://imgserver.10.68.0.250.nip.io/data/andra


In [3]:
pd.__version__

'1.4.3'

# Notebook to integrate the test/work out the clickclick algorithms

In [4]:
if os.path.isfile('./BRISQUE_data.pkl') == True:
    print('Good to go')

Good to go


In [5]:
df = pd.read_pickle('./BRISQUE_data.pkl')
df = df.reset_index(drop=True)
dfna = df.dropna().copy()

## Are `csv` files on the server?

In [6]:
os.environ['no_proxy']='10.68.0.250.nip.io'
__SERVER_URL__ = 'http://imgserver.10.68.0.250.nip.io'
__ROOT_PATH__ ='/data/andra'
url = __SERVER_URL__ + "/list"
header = {'img_path': __ROOT_PATH__}
response = requests.get(url, headers=header)
files = response.json()

In [7]:
if 'xl' in files:
    print('true')

## Nope, need to read the Excel files and get the useful information out of them

In [8]:
fileName1 = '../util/calci.csv'
df1 = pd.read_csv(fileName1,sep=';',encoding='cp1252', index_col=False)

In [9]:
fileName2 = '../util/result.csv'
df2 = pd.read_csv(fileName2,sep=';',encoding='cp1252', index_col=False)

In [10]:
df1

,Cote,1 minte,3 minutes,15 minutes,% de calci-dolomimÃ©trie,% CaCO3,% Dolomie,% insolubles
0,"0,25",15,19,21,15,6,79,21
1,"0,50",12,16,18,12,6,82,18
2,"0,75",14,18,20,14,6,80,20
3,"1,00",24,28,31,24,7,69,31
4,"1,25",19,22,25,19,6,75,25
...,...,...,...,...,...,...,...,...
118,"30,00",36,39,42,36,6,58,42
119,"30,10",70,71,72,70,2,28,72
120,"30,25",32,35,37,32,5,63,37
121,"30,50",39,41,43,39,4,57,43


This is the expected format. 

In [11]:
sery0 = pd.Series(dtype='str')
sery1 = pd.Series(dtype='str')
sery2 = pd.Series(dtype='str')

for i, j in df2.iterrows():
    cote0,cote1 = int(round(float(j["Cote0"]))),int(round(float(j["Cote1"])))
    s0,s1,s2 = "{","{","{"
    for k1, k2 in df1.iterrows():
        c = int(round(float(str(k2["Cote"]).replace(',','.'))*100))
        if cote0<=c and c<=cote1:
            if len(s0)!=1 : 
                s0+= ";"
                s1+= ";"
                s2+= ";"
            s0+= str(c)
            s1+= str(df1.iloc[k1, 1]) #str(k2["% calcimétrie 1 min "]) #
            s2+= str(df1.iloc[k1, 3]) #str(k2["% calcimétrie 15 min"]) # 
    s0+= "}"
    s1+= "}"
    s2+= "}"
    sery0.at[i] = s0
    sery1.at[i] = s1
    sery2.at[i] = s2

df2["CalciCotes"]= sery0
df2["CalciVals1m"]= sery1
df2["CalciVals15m"]= sery2

In [16]:
fileName1 = '/work/armitagj/data/Données Calci & Photos/BPE4023/GTR2012-07_0023_BPE4023_Calcimétrie.xls'
df1 = pd.read_excel(fileName1)
df1

,Unnamed: 0,Laboratoire de Recherche Souterrain de Meuse/Haute-Marne,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,SUIVI GEOLOGIQUE DES FORAGES DES EXPERIMENTATI...,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GEOTER,Forage n° BPE4023,NaN,NaN,NaN,NaN,NaN
3,FT31b,NaN,NaN,NaN,NaN,NaN,NaN
4,Galerie GRD,NaN,NaN,Date début : 02/08/12,NaN,Date fin : 02/08/12,NaN
5,Mesures de calci-dolomimétrie,NaN,NaN,NaN,NaN,NaN,NaN
6,Cote,% de calci-dolomimétrie,NaN,NaN,% CaCO3,% Dolomie,% insolubles
7,NaN,à 1 minute,à 4 minutes,à 19 minutes,NaN,NaN,NaN
8,0.56,29,31,33,29,4,67
9,0.65,31,33,33,31,2,67


In [17]:
df = df1.dropna().copy()
df.columns = ['Cote', 'à 1 minute', 'à 4 minutes', 'à 19 minutes', '% CaCO3', '% Dolomie', '% insolubles']
df

,Cote,à 1 minute,à 4 minutes,à 19 minutes,% CaCO3,% Dolomie,% insolubles
8,0.56,29,31,33,29,4,67
9,0.65,31,33,33,31,2,67
10,0.75,33,35,35,33,2,65
11,0.85,33,35,36,33,3,64
12,0.95,33,33,36,33,3,64
13,1.05,32,32,33,32,1,67
14,1.15,32,33,33,32,1,67
15,1.25,32,33,33,32,1,67
16,1.34,32,32,33,32,1,67
17,1.44,32,33,33,32,1,67


In [66]:
directories = []
for name in os.listdir('../database/excell-files/'):
    if os.path.isdir('../database/excell-files/' + name):
        directories.append('../database/excell-files/' + name)

In [67]:
excell_files = []
for directory in directories:
    for fname in os.listdir(directory):    # change directory as needed
        if os.path.isfile(f'{directory}/{fname}'):
            if 'Calcimétrie' in fname and 'Zone.Identifier' not in fname:
                excell_files.append(f'{directory}/{fname}')
            elif 'CALCIMETRIE' in fname and 'Zone.Identifier' not in fname:
                excell_files.append(f'{directory}/{fname}')
            elif 'Calcimètre' in fname and 'Zone.Identifier' not in fname:
                excell_files.append(f'{directory}/{fname}')
            elif 'Calci' in fname and 'Zone.Identifier' not in fname:
                excell_files.append(f'{directory}/{fname}')
            elif 'calcimétrie' in fname and 'Zone.Identifier' not in fname:
                excell_files.append(f'{directory}/{fname}')

In [68]:
len(directories)

86

In [69]:
len(excell_files)

86

In [115]:
cote = []
dfcolms = []
for file in excell_files:
    df = pd.read_excel(file)
    sf = df.iloc[:, 0]
    if 'Cote' in list(sf):
        cote.append(sf[sf == 'Cote'].index[0])
        dfcolms.append(len(df.columns))
                
        if len(df.columns) > 7:           
            for i in range(7, len(df.columns)):
                colname = f'Unnamed: {i}'
                df = df.drop([colname], axis=1)
        
        dfe = df.dropna().copy()
        dfe.columns = ['Cote', 'à 1 minute', 'à 4 minutes', 'à 19 minutes', '% CaCO3', '% Dolomie', '% insolubles']
    
    if 'Cote toit' in list(sf):
        cote.append(sf[sf == 'Cote toit'].index[0])
        dfcolms.append(len(df.columns))
        
        if len(df.columns) > 7:           
            for i in range(7, len(df.columns)):
                colname = f'Unnamed: {i}'
                df = df.drop([colname], axis=1)
        
        dfe = df.dropna().copy()
        df.drop(df.columns[1], axis=1)
        dfe.columns = ['Cote', 'à 1 minute', 'à 3 minutes', 'à 15 minutes', '% CaCO3', '% Dolomie', '% insolubles']
        

In [117]:
dfe

,Cote,à 1 minute,à 4 minutes,à 19 minutes,% CaCO3,% Dolomie,% insolubles
8,4.85,25,27,29,25,4,71
9,6.7,25,27,29,25,4,71
10,8.6,25,27,29,25,4,71
11,10.45,25,27,29,25,4,71
12,12.51,27,27,29,27,2,71
13,14.45,25,27,29,25,4,71
14,16.55,25,27,29,25,4,71
15,18.38,27,29,31,27,4,69
16,21.27,20,22,25,20,5,75
17,24.38,27,27,29,27,2,71


In [142]:
def excell2csv(filelocs):
    directories = []
    for name in os.listdir(filelocs):
        if os.path.isdir(filelocs + name):
            directories.append(filelocs + name)
    
    excell_files = []
    for directory in directories:
        for fname in os.listdir(directory):
            if os.path.isfile(f'{directory}/{fname}'):
                # different file names of course
                if 'Calcimétrie' in fname and 'Zone.Identifier' not in fname:
                    excell_files.append(f'{directory}/{fname}')
                elif 'CALCIMETRIE' in fname and 'Zone.Identifier' not in fname:
                    excell_files.append(f'{directory}/{fname}')
                elif 'Calcimètre' in fname and 'Zone.Identifier' not in fname:
                    excell_files.append(f'{directory}/{fname}')
                elif 'Calci' in fname and 'Zone.Identifier' not in fname:
                    excell_files.append(f'{directory}/{fname}')
                elif 'calcimétrie' in fname and 'Zone.Identifier' not in fname:
                    excell_files.append(f'{directory}/{fname}')
    
    dfcolms = []
    for file in excell_files:
        df = pd.read_excel(file)
        sf = df.iloc[:, 0]
        if 'Cote' in list(sf):
            dfcolms.append(len(df.columns))
            
            if len(df.columns) < 5:
                print(file)
                break

            if len(df.columns) > 7:           
                for i in range(7, len(df.columns)):
                    colname = f'Unnamed: {i}'
                    df = df.drop([colname], axis=1)

            dfe = df.dropna().copy()
            dfe.columns = ['Cote', 'à 1 minute', 'à 4 minutes', 'à 19 minutes', '% CaCO3', '% Dolomie', '% insolubles']

        if 'Cote toit' in list(sf):
            dfcolms.append(len(df.columns))
            
            if len(df.columns) < 5:
                print(file)
                break

            if len(df.columns) > 7:           
                for i in range(7, len(df.columns)):
                    colname = f'Unnamed: {i}'
                    df = df.drop([colname], axis=1)

            dfe = df.dropna().copy()
            df.drop(df.columns[1], axis=1)
            dfe.columns = ['Cote', 'à 1 minute', 'à 3 minutes', 'à 15 minutes', '% CaCO3', '% Dolomie', '% insolubles']
    
        csv_file = file.split('.xls')[0] + '.csv'
        print(csv_file)
        dfe.to_csv(csv_file)

In [143]:
excell2csv('../database/excell-files/')

../database/excell-files/BPE4023/GTR2012-07_0023_BPE4023_Calcimétrie.csv
../database/excell-files/BPE4024/GTR2012-07_0023_BPE4024_Calcimétrie.csv
../database/excell-files/DCN2001/GTR2013-10_0026_DCN2001_Calcimétrie.csv
../database/excell-files/DCN2002/GTR2013-10_0026_DCN2002_Calcimétrie.csv
../database/excell-files/DCN2003/GTR2013-10_0026_DCN2003_Calcimétrie.csv
../database/excell-files/DCN2006/GTR2016-09_0022_DCN2006_Calcimétrie.csv
../database/excell-files/DIR1001/GTR2005-01_0055_DIR1001_CALCIMETRIE.csv
../database/excell-files/DIR1002/GTR2005-10_0053_DIR1002_CALCIMETRIE.csv
../database/excell-files/DIR1003/GTR2005-10_0049_DIR1003_CALCIMETRIE.csv
../database/excell-files/DIR2001/GTR2005-01_0012_DIR2001_CALCIMETRIE.csv
../database/excell-files/DIR2002/GTR2005-01_0011_DIR2002_CALCIMETRIE.csv
../database/excell-files/DIR2003/GTR2005-01_0009_DIR2003_CALCIMETRIE.csv
../database/excell-files/DPC3015/GTR2016-03_0006_DPC3015_Calcimétrie.csv
../database/excell-files/DPC3022/GTR2014-02_0003_DP

In [136]:
df

,Unnamed: 0,Laboratoire de Recherche Souterrain de Meuse/Haute-Marne,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,SUIVI GEOLOGIQUE DES FORAGES DES EXPERIMENTATI...,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GEOTER,Forage OHZ3005,NaN,NaN,NaN,NaN,NaN
3,FT31a,NaN,NaN,NaN,NaN,NaN,NaN
4,Galerie : GRM,NaN,NaN,Date début : 18/03/2016,NaN,Date fin : 18/03/2016,NaN
5,Mesures de calci-dolomimétrie,NaN,NaN,NaN,NaN,NaN,NaN
6,Cote,% de calci-dolomimétrie,NaN,NaN,% CaCO3,% Dolomie,% insolubles
7,NaN,à 1 minute,à 3 minutes,à 15 minutes,NaN,NaN,NaN
8,4.85,25,27,29,25,4,71
9,6.7,25,27,29,25,4,71


In [133]:
len(df.columns)

7

In [139]:
df = pd.read_excel(file)

In [141]:
if len(df.columns) < 7:
    print('true')